In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from time import sleep
import datetime
from webdriver_manager.chrome import ChromeDriverManager
import pickle
from random import randint

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import UnexpectedAlertPresentException

In [2]:
df1 = pd.read_pickle("Nicholas' Data")
df2 = pd.read_csv("Jacky1.csv")
df3 = pd.read_csv("Jacky2.csv")
df4 = pd.read_csv("ZhouWei.csv")

print(f"""
length of df1: {len(df1)}
length of df2: {len(df2)}
length of df3: {len(df3)}
length of df4: {len(df4)}
""")

overall_data = pd.concat([df1,df2,df3,df4], ignore_index= True)
overall_data = overall_data.drop_duplicates(subset = 'Stall Name',  ignore_index= True)


length of df1: 12661
length of df2: 5663
length of df3: 32649
length of df4: 46826



In [3]:
infile1 = open("scraped locations.pkl", "rb")
test = pickle.load(infile1)
infile1.close()

In [4]:
merging_list = test

In [5]:
len(merging_list)

11851

In [6]:
overall_data.loc[10162]

Stall Name          Teochew Kway Chap (Amoy Street Food Centre)
Cuisine                                      [Chinese, Teochew]
Shop Type                                        Kiosk or Stall
Address       #02-117 Amoy Street Food Centre Maxwell Road 0...
Unnamed: 0                                                  NaN
Name: 10162, dtype: object

In [7]:
merging_list[10162]

['7 Maxwell Rd, #02-117, Singapore 069111', 4.8, 18]

In [8]:
google_query = 'https://www.google.com/search?source=hp&ei=RMtRYMaDCML69QPxvYzQBQ&iflsig=AINFCbYAAAAAYFHZVCGjKdQ7SCTBw9cWYUrAyDMsHWCs&q='

In [9]:
len(merging_list)

11851

In [10]:
len(overall_data)

20849

In [11]:
copy = merging_list

In [12]:
copy

[['6 Jalan Bukit Merah, #01-120, Singapore 150006', 4.2, 153],
 ['Hindoo Rd', 4.3, 435],
 [None, None, None],
 ['6 Jalan Bukit Merah, #01-106, Singapore 150006', 5.0, 1],
 ['20 Jln Pari Burong, Singapore 488686', 4.2, 34],
 ['33 Sengkang West Ave, #B1-K7/K8/K9, Singapore 797653', 3.8, 8],
 ['247 Jln Kayu, Singapore 799471', 3.8, 362],
 ['33 Sengkang West Ave, Singapore 797653', 4.4, 635],
 ['10 Bukit Chermin Rd, M Level, 109918', 3.9, 445],
 ['33 Sengkang West Ave, #B1-26 The Seletar Mall, Singapore 797653', 4.1, 38],
 ['6 Jalan Bukit Merah, #01-09, Singapore 150006', None, 38],
 [None, None, None],
 ['2 Pahang St, Singapore 198604', 4.1, 122],
 ['144 Upper Bukit Timah Rd, Singapore 588177', 5.0, 1],
 ['44 Jln Merah Saga, #01-52 Chip Bee Gardens, Singapore 278116', 4.1, 496],
 ['4A Jln Batu, Singapore 430002', None, 496],
 [None, None, None],
 ['223 Jln Kayu, Singapore 799447', 4.0, 40],
 ['252 Jurong East St 24', None, 40],
 ['44 Jln Merah Saga, Singapore 278116', 4.1, 378],
 ['275 Jl

In [13]:
item = (merging_list).pop()

In [14]:
item

['33 Lor Liput, Singapore 277744', 3.5, 40]

In [15]:
copy[:5]

[['6 Jalan Bukit Merah, #01-120, Singapore 150006', 4.2, 153],
 ['Hindoo Rd', 4.3, 435],
 [None, None, None],
 ['6 Jalan Bukit Merah, #01-106, Singapore 150006', 5.0, 1],
 ['20 Jln Pari Burong, Singapore 488686', 4.2, 34]]

In [16]:
overall_data.loc[20844]

Stall Name                       Snackz It! (Vivo City)
Cuisine                    ['Asian', 'Local', 'Snacks']
Shop Type                       Confectionery or Bakery
Address       #B2-K24 Vivocity Harbourfront Walk 098585
Unnamed: 0                                        46744
Name: 20844, dtype: object

In [17]:
reviews = pd.DataFrame(data = copy, columns = ["Google Address", "Average Review", "Number of Reviews"])

In [18]:
df = pd.concat([overall_data,reviews], axis = 1)

In [19]:
df3 = pd.DataFrame(df['Cuisine'])

In [20]:
df3['Cuisine 1'] = None
df3['Cuisine 2'] = None
df3['Cuisine 3'] = None

In [21]:
(df3.loc[20000]["Cuisine"])

"['Asian', 'Chinese', 'Hainanese', 'Local', 'Singaporean']"

In [22]:
for index, cuisine_list in df3.iterrows():
    try:
        if(isinstance(cuisine_list["Cuisine"], str)):
            cuisine = literal_eval(cuisine_list["Cuisine"])
        else:
            cuisine = cuisine_list["Cuisine"]
        
        df3.loc[index, "Cuisine 1"] = cuisine[0]
        df3.loc[index, "Cuisine 2"] = cuisine[1]
        df3.loc[index, "Cuisine 3"] = cuisine[2]
    
    except:
        continue

In [23]:
from ast import literal_eval

literal_eval(df3.loc[20844]["Cuisine"])

['Asian', 'Local', 'Snacks']

In [24]:
df2 = df3.drop(columns="Cuisine")

In [25]:
final_df = pd.concat([df,df2], axis = 1)

In [26]:
final_df.columns

Index(['Stall Name', 'Cuisine', 'Shop Type', 'Address', 'Unnamed: 0',
       'Google Address', 'Average Review', 'Number of Reviews', 'Cuisine 1',
       'Cuisine 2', 'Cuisine 3'],
      dtype='object')

In [27]:
final_df = final_df[['Stall Name', 'Cuisine','Cuisine 1', 'Cuisine 2',
       'Cuisine 3', 'Shop Type', 'Address', 'Google Address',
       'Average Review', 'Number of Reviews']]

In [28]:
final_df.to_csv("SG Food Stalls.csv", index = False)

In [29]:
df = pd.read_csv("SG Food Stalls.csv")

In [30]:
missing_data = df[df["Google Address"].isna()]

In [31]:
missing_data

,Stall Name,Cuisine,Cuisine 1,Cuisine 2,Cuisine 3,Shop Type,Address,Google Address,Average Review,Number of Reviews
2,Stall #01-46,"['Chinese', 'Snacks']",Chinese,Snacks,NaN,Kiosk or Stall,#01-46 Jalan Batu Food Centre Jalan Batu 432004,NaN,NaN,NaN
11,Soon Huat Satay Mee Hoon,"['Asian', 'Chinese']",Asian,Chinese,NaN,Kiosk or Stall,#01-47 Abc Brickworks Market And Food Centre J...,NaN,NaN,NaN
16,Amoy Street Famous 16 Prawn Noodle,['Chinese'],Chinese,NaN,NaN,Kiosk or Stall,#01-126 ABC Brickworks Food Centre Jalan Bukit...,NaN,NaN,NaN
24,Xiang Garden Restaurant,"['Asian', 'Chinese']",Asian,Chinese,NaN,Restaurant,Jalan Kayu 799472,NaN,NaN,NaN
26,Brew & Bar,"['Asian', 'Nonya / Peranakan']",Asian,Nonya / Peranakan,NaN,Restaurant,Jalan Pari Burong 488672,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
20844,Snackz It! (Vivo City),"['Asian', 'Local', 'Snacks']",NaN,NaN,NaN,Confectionery or Bakery,#B2-K24 Vivocity Harbourfront Walk 098585,NaN,NaN,NaN
20845,LiHO (Wisma Atria),"['Drinks', 'Juices', 'Singaporean']",NaN,NaN,NaN,Kiosk or Stall,#B1-03 Wisma Atria Orchard Road 238877,NaN,NaN,NaN
20846,Taste of Thailand,"['Asian', 'Indochinese', 'Thai']",NaN,NaN,NaN,Restaurant,#01-1001 Yishun Industrial Food Centre Yishun ...,NaN,NaN,NaN
20847,Mr Bean (Khoo Teck Puat Hospital),"['Desserts', 'Drinks', 'Snacks']",NaN,NaN,NaN,Kiosk or Stall,#01-08 Khoo Teck Puat Hospital Yishun Central ...,NaN,NaN,NaN


### Scrape Missing Data

In [32]:
pd.read_csv("Final_list.csv")

,placeId,Stall Name,Cuisine,Cuisine 1,Cuisine 2,Cuisine 3,Shop Type,Address,Google Address,Average Review,Number of Reviews
0,0,Fatty Cheong,[None],NaN,NaN,NaN,Kiosk or Stall,#01-144 ABC Brickworks Food Centre Jalan Bukit...,"6 Jalan Bukit Merah, #01-120, Singapore 150006",4.2,153.0
1,1,Mani Iyer Mess,['South Indian'],South Indian,NaN,NaN,Restaurant,#01-01 Little India Conservation Area Hindoo R...,Hindoo Rd,4.3,435.0
2,2,Stall #01-46,"['Chinese', 'Snacks']",Chinese,Snacks,NaN,Kiosk or Stall,#01-46 Jalan Batu Food Centre Jalan Batu 432004,NaN,NaN,NaN
3,3,Wei Xiang Kitchen,"['Chinese', 'Seafood']",Chinese,Seafood,NaN,Kiosk or Stall,#01-106 ABC Brickworks Food Centre Jalan Bukit...,"6 Jalan Bukit Merah, #01-106, Singapore 150006",5.0,1.0
4,4,The Thai Table,"['Asian', 'Indochinese', 'Thai']",Asian,Indochinese,Thai,Restaurant,Picardy Garden Jalan Pari Burong 488686,"20 Jln Pari Burong, Singapore 488686",4.2,34.0
...,...,...,...,...,...,...,...,...,...,...,...
20844,20844,Snackz It! (Vivo City),"['Asian', 'Local', 'Snacks']",Asian,Local,Snacks,Confectionery or Bakery,#B2-K24 Vivocity Harbourfront Walk 098585,"1 HarbourFront Walk, #B2-K24 VivoCity, Singapo...",2.0,3.0
20845,20845,LiHO (Wisma Atria),"['Drinks', 'Juices', 'Singaporean']",Drinks,Juices,Singaporean,Kiosk or Stall,#B1-03 Wisma Atria Orchard Road 238877,"#B1-03, Wisma Atria, 435 Orchard Road, 238877",3.7,65.0
20846,20846,Taste of Thailand,"['Asian', 'Indochinese', 'Thai']",Asian,Indochinese,Thai,Restaurant,#01-1001 Yishun Industrial Food Centre Yishun ...,"1001 Yishun Industrial Park A, Singapore 768743",4.1,258.0
20847,20847,Mr Bean (Khoo Teck Puat Hospital),"['Desserts', 'Drinks', 'Snacks']",Desserts,Drinks,Snacks,Kiosk or Stall,#01-08 Khoo Teck Puat Hospital Yishun Central ...,NaN,NaN,NaN
